In [390]:
import re
import time
import datetime
from datetime import datetime
import urllib
import tldextract
import ipaddress
import requests
import bs4
from bs4 import BeautifulSoup
import dns.resolver
import whois

In [391]:
url = 'https://www.gitam.edu/'

In [392]:
urlinfo = urllib.parse.urlsplit(url)
netloc = urlinfo.netloc
domain = '.'.join(netloc.split('.')[-2:])
domain_name = tldextract.extract(url).domain
path = urlinfo.path

netloc, domain, domain_name, path

('www.gitam.edu', 'gitam.edu', 'gitam', '/')

In [393]:
def urlLength(url):
    return len(url)

In [394]:
def havingIPAddress(domain):
  try:
    ipaddress.ip_address(domain)
    ip = 1
  except:
    ip = 0
  return ip

In [395]:
def countDots(netloc):
    return netloc.count('.')

In [396]:
def countHyphens(url):
    return url.count('-')

In [397]:
def countATR(url):
     return url.count('@')

In [398]:
def countSlash(url):
    return url.count('/')

In [399]:
def countDoubleSlash(url):
    list=[x.start(0) for x in re.finditer('//', url)]
    if list[len(list)-1]>6:
        return 1
    else:
        return 0

In [400]:
def countHTTP(url):
    return url.count('http')

In [401]:
def ratioOfDigits(url):
    return len(re.sub("[^0-9]", "", url))/len(url)

In [402]:
def seperatedPrefixSuffix(url):
    if re.findall(r"https?://[^\-]+-[^\-]+/", url):
        return 1
    else:
        return 0 

In [403]:
def usingShorteningServices(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0

In [404]:
try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    count=0
    for i in soup.find_all('a'):
        count = count+1
    num_hyperlinks = count
except:
    num_hyperlinks = 0

In [405]:
#To get all the page HTML content
try:
    response = requests.get(url)
    page_content = response.content
except:
    page_content = ""

In [406]:
#To get all the text content from the HTML
soup = BeautifulSoup(page_content, 'html.parser', from_encoding='iso-8859-1')
content_text = soup.get_text()

In [407]:
def usingIFrame(page_content):
  if page_content == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", str(page_content)):
          return 0
      else:
          return 1

In [408]:
def disabledRightClick(page_content):
  if page_content == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", str(page_content)):
      return 1
    else:
      return 0

In [409]:
def domainWithCopyright(domain_name, content_text):
    try:
        m = re.search(u'(\N{COPYRIGHT SIGN}|\N{TRADE MARK SIGN}|\N{REGISTERED SIGN})', content_text)
        _copyright = content_text[m.span()[0]-50:m.span()[0]+50]
        if domain_name.lower() in _copyright.lower():
            return 0
        else:
            return 1
    except:
        return 1

In [410]:
def registeredWHOIS(netloc):
    try:
        hostname = whois.whois(netloc).domain_name
        if type(hostname) == list:
            for host in hostname:
                if re.search(host.lower(), netloc):
                    return 0
            return 1
        else:
            if re.search(hostname.lower(), netloc):
                return 0
            else:
                return 1     
    except:
        return 1

In [411]:
def domainRegisterationLength(netloc):
    try:
        response = whois.whois(netloc)
        expiration_date = response.expiration_date
        today = time.strftime('%Y-%m-%d')
        today = datetime.strptime(today, '%Y-%m-%d')
        if expiration_date:
            if type(expiration_date) == list:
                expiration_date = min(expiration_date)
            return abs((expiration_date - today).days)
        else:
            return 0
    except:
        return -1

In [412]:
domainageAPI = "https://endpoint.apivoid.com/domainage/v1/pay-as-you-go/?key=c1b80285225711816ec3ff48c7c43a58b970e0e3&host="+hostname
res = requests.get(url = domainageAPI)
data = res.json()
try:
    domain_age = data['data']['domain_age_in_days']
except:
    domain_age = -1

In [413]:
def webTraffic(url):
        try:
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        except:
            return 0
        return int(rank)

In [414]:
def havingDNSRecords(domain):
    try:
        nameservers = dns.resolver.resolve(domain,'NS')
        if len(nameservers)>0:
            return 0
        else:
            return 1
    except:
        return 1

In [415]:
def googleIndex(url):
    google = "https://www.google.com/search?q=site:" + url + "&hl=en"
    response = requests.get(google, cookies={"CONSENT": "YES+1"})
    soup = BeautifulSoup(response.content, "html.parser")
    not_indexed = re.compile("did not match any documents")
    if soup(text=not_indexed):
        return 1
    else:
        return 0

In [416]:
pagerankAPI = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D='+domain
key = 's8wsc84cgcckckkscg4c04k48s4s8k08w0k0gokk'
res = requests.get(url = pagerankAPI, headers={'API-OPR':key})
data = res.json()
page_rank = data['response'][0]['page_rank_integer']

In [417]:
feature_array = []
feature_array.append(urlLength(url))
feature_array.append(havingIPAddress(domain))
feature_array.append(countDots(netloc))
feature_array.append(countHyphens(url))
feature_array.append(countATR(url))
feature_array.append(countSlash(url))
feature_array.append(countDoubleSlash(url))
feature_array.append(countHTTP(url))
feature_array.append(ratioOfDigits(url))
feature_array.append(seperatedPrefixSuffix(url))
feature_array.append(usingShorteningServices(url))
feature_array.append(num_hyperlinks)
feature_array.append(usingIFrame(page_content))
feature_array.append(disabledRightClick(page_content))
feature_array.append(domainWithCopyright(domain_name, content_text))
feature_array.append(registeredWHOIS(netloc))
feature_array.append(domainRegisterationLength(netloc))
feature_array.append(domain_age)
feature_array.append(webTraffic(url))
feature_array.append(havingDNSRecords(domain))
feature_array.append(googleIndex(url))
feature_array.append(page_rank)
feature_array

[22, 0, 2, 0, 0, 3, 0, 1, 0.0, 0, 0, 282, 0, 0, 0, 0, 0, -1, 0, 0, 0, 5]